### Apply traditional alogorithms to classify data

In [ ]:
# -*- coding: utf-8 -*-

import laspy
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("../src")
from preprocessing.summary_stats import summarize_las

### Summary stats

In [ ]:
# load file .las
las = laspy.read("../data/processed/raw_data_downsampled_1m.las")

summarize_las(las)

In [ ]:
# load file .las
las = laspy.read("../data/processed/raw_data_cleaned_1m.las")

summarize_las(las)

In [ ]:
# load file .las
las = laspy.read("../data/processed/raw_data_cleaned_20m.las")

summarize_las(las)

### Ground filter

In [ ]:
from traditional_methods.csf_wrapper import run_csf

# define input, output
input_path = "../data/raw/raw_data_tscan.las"
output_path = "../data/processed/ground_filtered_csf.las"

# CSF params
csf_params = {
    "rigidness": 2,
    "step": 0.65,
}

output_file_csf = run_csf(
    input_file=input_path,
    output_file=output_path,
    **csf_params
)

In [ ]:
las = laspy.read(output_file_csf)
summarize_las(las)

In [ ]:
from traditional_methods.pmf_wrapper import run_pmf

# define input, output
input_path = "../data/raw/raw_data_tscan.las"
output_path = "../data/processed/ground_filtered_pmf.las"

# PMF params
pmf_params = {
    "max_window_size": 33,
    "slope": 1.0,
    "initial_distance": 0.5,
    "cell_size": 1.0,
}

output_file_pmf = run_pmf(
    input_file=input_path,
    output_file=output_path,
    **pmf_params
)

In [ ]:
las = laspy.read(output_file_pmf)
summarize_las(las)

In [ ]:
from traditional_methods.smrf_wrapper import run_smrf

# define input, output
input_path = "../data/processed/raw_data_tscan.las"
output_path = "../data/processed/ground_filtered_smrf.las"

# SMRF params
smrf_params = {
    "window": 18.0,
    "slope": 0.15,
    "threshold": 0.5,
    "window_increment": 16.0,
}

output_file_smrf = run_smrf(
    input_file=input_path,
    output_file=output_path,
    **smrf_params
)

In [ ]:
las = laspy.read(output_file_smrf)
summarize_las(las)

### Evalution Filter

In [ ]:
import laspy
import numpy as np
from evaluation.metrics import evaluate_classification

# load data
las_pred = laspy.read("../data/processed/ground_filtered_pmf.las")
las_gt   = laspy.read("../data/ground_truth/ground_truth.las")

# assuming classification is stored in the 'classification' attribute
# and both las_pred and las_gt have the same structure
pred_labels = las_pred.classification
gt_labels = las_gt.classification

# evaluate
results = evaluate_classification(pred_labels, gt_labels)
print(results)

In [ ]:
from scipy.spatial import cKDTree

# coordinates
coords_pred = np.vstack((las_pred.x, las_pred.y, las_pred.z)).T
coords_gt   = np.vstack((las_gt.x, las_gt.y, las_gt.z)).T

tree = cKDTree(coords_gt)
dists, idxs = tree.query(coords_pred, k=1)

# threshold: distance to accept the match (es. 0.5m)
valid = dists < 0.5
pred_labels = las_pred.classification[valid]
gt_labels = las_gt.classification[idxs[valid]]

# Evaluation
results = evaluate_classification(pred_labels, gt_labels)

In [ ]:
import open3d as o3d
import numpy as np

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(np.vstack((las.x, las.y, las.z)).T)

# Color by elevation
z_norm = (las.z - las.z.min()) / (las.z.max() - las.z.min())
pcd.colors = o3d.utility.Vector3dVector(plt.cm.viridis(z_norm)[:, :3])

o3d.visualization.draw_geometries([pcd])